# Обработка данных по основным параметрам прогнозов СЭР для передачи данных в регрессионную модель

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as matplotlib
print(matplotlib.__version__)

3.3.4


In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', 100)

In [3]:
FIGSIZE = (12,5)

Параметры выбраны в ручную, так как в каждом годе перечень отличается

In [4]:
FEATURES = ['Численность постоянного населения',
            'Индекс  потребительских цен',
            'Валовой региональный продукт',
            'Индекс промышленного производства',
            'Добыча полезных ископаемых',
            'Обрабатывающие производства',
            'Продукция сельского хозяйства',
            'Производство и распределение электроэнергии, газа и воды',
            'железорудные окатыши',
            'щебень и гравий',
            'древесина необработанная',
            'лесоматериалы продольно распиленные',
            'древесностружечные плиты, плиты OSB',
            'бумага',
            'целлюлоза древесная и целлюлоза из прочих волокнистых материалов (сульфатная, сульфитная)',
            'мешки бумажные',
            'Картофель',
            'Овощи',
            'Скот и птица',
            'Молоко',
            'Яйца',
            'Реализация алкогольной продукции организациями -производителями',
            'Инвестиции в основной капитал',
            'Ввод в эксплуатацию жилых домов',
            'Оборот розничной торговли',
            'Объем платных услуг населению',
            'Фонд заработной платы',
            'Реальные денежные доходы населения',
            'Прибыль прибыльных организаций',
            'Налогооблагаемая прибыль',
            'Среднегодовая остаточная стоимость облагаемого имущества',
            'Внешнеторговый оборот',
            'Экспорт',
            'Импорт',
            'Численность безработных, зарегистрированных в службах занятости',
            'Уровень зарегистрированной безработицы (к численности экономически активного населения)']

In [5]:
AGE_BEGIN = 2010
AGE_END = 2020
years = [year for year in range(AGE_BEGIN,AGE_END+1)]

Чтение данных по прогнозам СЭР

In [6]:
from pathlib import Path
import re

PATH = '../data/Основные параметры прогнозов СЭР/'

directory = Path(PATH)

sed = {}
l = list(directory.iterdir())

for year in years:
    for x in directory.iterdir(): # bad O
        if re.match(f'{year}-{year+2}', x.name):
            df = pd.read_excel(f'{PATH}{x.name}', header=1)
            sed[year] = df
            break

In [7]:
for year in years:
    sed[year].rename(columns={f'{sed[year].columns[0]}':'indicator'}, inplace=True)

In [ ]:
FEATURES[1].replace(" ",'') in sed[2015]['indicator'].str.replace(" ",'').iloc[-1]

Формирование датафрейма

In [9]:
features = []

for feature in FEATURES:
    #print(feature)
    forecast = []
    for year in years:
        #print(year)
        try:
            forecast.append(sed[year][sed[year]['indicator'].str.replace(" ",'').str.lower().str.contains(feature.replace(" ",'').lower(), regex=False).fillna(False)].iloc[:,-3].values[0])
        except:
            forecast.append(np.nan)
    try:
        forecast.append(sed[years[-1]][sed[years[-1]]['indicator'].str.replace(" ",'').str.lower().str.contains(feature.replace(" ",'').lower(), regex=False).fillna(False)].iloc[:,-2].values[0])
    except:
        forecast.append(np.nan)
    try:
        forecast.append(sed[years[-1]][sed[years[-1]]['indicator'].str.replace(" ",'').str.lower().str.contains(feature.replace(" ",'').lower(), regex=False).fillna(False)].iloc[:,-1].values[0])
    except:
        forecast.append(np.nan)
    #print(tmp)
    features.append(forecast)

Добавим данные СЭР для 2021 и 2022 годов

In [11]:
dic = {"Год": [*years, 2021, 2022]}
dic.update({f'{f}': feature for (f, feature) in zip(FEATURES, features)})

In [12]:
df = pd.DataFrame(dic)
df

,Год,Численность постоянного населения,Индекс потребительских цен,Валовой региональный продукт,Индекс промышленного производства,Добыча полезных ископаемых,Обрабатывающие производства,Продукция сельского хозяйства,"Производство и распределение электроэнергии, газа и воды",железорудные окатыши,щебень и гравий,древесина необработанная,лесоматериалы продольно распиленные,"древесностружечные плиты, плиты OSB",бумага,"целлюлоза древесная и целлюлоза из прочих волокнистых материалов (сульфатная, сульфитная)",мешки бумажные,Картофель,Овощи,Скот и птица,Молоко,Яйца,Реализация алкогольной продукции организациями -производителями,Инвестиции в основной капитал,Ввод в эксплуатацию жилых домов,Оборот розничной торговли,Объем платных услуг населению,Фонд заработной платы,Реальные денежные доходы населения,Прибыль прибыльных организаций,Налогооблагаемая прибыль,Среднегодовая остаточная стоимость облагаемого имущества,Внешнеторговый оборот,Экспорт,Импорт,"Численность безработных, зарегистрированных в службах занятости",Уровень зарегистрированной безработицы (к численности экономически активного населения)
0,2010,684.00,111.00,125000,98.00,100.00,NaN,NaN,95.00,6000,10500.00,NaN,NaN,NaN,955,NaN,300,77.50,22.80,12.50,71.20,54.90,NaN,17000,140.00,61300,18200,57500,96.00,2500.00,3500,95500,1235.00,1025.00,210.00,13.40,3.50
1,2011,680.50,106.50,118500,100.00,96.00,101.50,101.50,98.20,9000,8000.00,5500.00,690.00,80.00,980,1085.00,315,71.50,23.00,12.60,72.00,64.00,NaN,22920,135.00,62400,18500,57300,99.00,14000.00,20400,94300,1462.00,1222.00,240.00,13.10,3.50
2,2012,639.00,107.25,159000,101.80,102.00,102.00,96.00,100.00,10100,11000.00,5800.00,730.00,100.00,1015,1105.00,360,70.50,20.00,12.50,59.00,10.00,NaN,29320,179.00,76300,20400,63300,101.00,19700.00,22000,99000,1930.00,1580.00,350.00,10.50,2.82
3,2013,636.10,105.50,16300,101.50,101.50,101.00,100.70,102.40,10200,15000.00,5300.00,640.00,160.00,1000,1105.00,370,71.60,19.00,13.00,65.60,8.40,300.00,34220,202.80,85000,25650,71900,102.00,5500.00,21350,101000,2035.00,1530.00,505.00,8.70,2.40
4,2014,633.50,105.50,154300,100.00,100.00,98.00,NaN,100.00,10400,17500.00,5500.00,670.00,145.00,720,850.00,380,69.00,18.10,12.60,61.00,3.00,270.00,38270,205.00,95000,27600,81200,100.00,18000.00,14000,109500,1365.00,1120.00,245.00,7.70,2.33
5,2015,631.70,106.50,192000,100.50,100.00,100.50,NaN,99.00,10500,19200.00,6000.00,720.00,300.00,800,900.00,390,82.50,19.20,12.10,65.60,4.00,470.00,35270,220.00,101000,29800,81700,99.00,19300.00,12200,158000,1447.00,1257.00,190.00,7.70,2.41
6,2016,628.40,107.50,207100,100.00,100.00,100.00,NaN,99.00,10580,13600.00,6100.00,735.00,345.00,865,940.00,425,90.00,19.00,12.00,66.40,5.00,5.00,35660,245.00,104000,33000,80500,98.00,NaN,18200,148000,975.00,820.00,155.00,7.80,2.40
7,2017,626.20,106.00,220700,102.00,101.50,102.50,NaN,101.50,10660,16000.00,6300.00,800.00,380.00,920,995.00,460,90.30,18.80,5.90,72.50,5.60,426.50,36500,250.00,110600,32800,81700,99.50,22500.00,23600,135800,810.00,680.00,130.00,8.40,2.59
8,2018,623.30,104.00,249900,102.00,101.50,102.00,NaN,NaN,10905,18800.00,7000.00,900.00,406.00,985,1035.00,470,77.00,17.50,6.00,65.50,5.50,310.50,41470,265.00,117000,33400,83900,100.20,NaN,27260,148700,1046.80,799.60,247.20,6.98,2.16
9,2019,616.70,NaN,285000,102.30,101.50,103.00,NaN,NaN,11100,18000.00,6800.00,920.00,406.00,1000,1035.00,464,70.00,17.60,4.90,64.00,5.65,158.00,46700,240.00,124600,36300,92600,100.00,NaN,40000,156000,1778.00,1570.00,208.00,6.10,1.95


Наблюдаем пропуски в данных причиной которых является отсутствие некоторых статей прогноза СЭР в некоторых годах

In [13]:
df.to_csv('../data/features.csv', index=False, encoding='utf-8-sig')